In [1]:
import re
import string
import numpy
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Masking
from keras.layers import Dense
from keras.layers import TimeDistributed

Using TensorFlow backend.


In [2]:
from google.colab import files
uploaded = files.upload()

Saving newMalTest.txt to newMalTest.txt
Saving newMalData.txt to newMalData.txt


In [0]:
with open('newMalData.txt') as f:
    arr = f.readlines()
    content = ''.join(arr)
    items = re.findall("^GET.*|POST.*$", content, re.MULTILINE)
    data = ""
    for item in items:
        temp_data = item.replace(" HTTP/1.1","").lower()
        data = data + temp_data + "\n"

In [0]:
chars = list(set(data + string.punctuation + string.ascii_lowercase + ' ' + string.digits))
VOCAB_SIZE = len(chars)
ix_to_char = {ix:char for ix, char in enumerate(chars)}
char_to_ix = {char:ix for ix, char in enumerate(chars)}
SEQ_LENGTH = 10
#print(chars)
DATA_LENGTH = int(len(data) / SEQ_LENGTH) 
#print(DATA_LENGTH, len(data))

In [0]:
X = numpy.zeros((DATA_LENGTH, SEQ_LENGTH, VOCAB_SIZE))
y = numpy.zeros((DATA_LENGTH, SEQ_LENGTH, VOCAB_SIZE))
for i in range(0, DATA_LENGTH):
    X_sequence = data[i*SEQ_LENGTH:(i+1)*SEQ_LENGTH]
    X_sequence_ix = [char_to_ix[value] for value in X_sequence]
    input_sequence = numpy.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        input_sequence[j][X_sequence_ix[j]] = 1.
    X[i] = input_sequence

    y_sequence = data[i*SEQ_LENGTH+1:(i+1)*SEQ_LENGTH+1]
    y_sequence_ix = [char_to_ix[value] for value in y_sequence]
    target_sequence = numpy.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        target_sequence[j][y_sequence_ix[j]] = 1.
    y[i] = target_sequence

In [0]:
model = Sequential()
model.add(LSTM(50, input_shape=(None, VOCAB_SIZE), return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [7]:
model.fit(X, y, batch_size=4, verbose=1, nb_epoch=1)

/usr/local/lib/python3.6/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
340172/340172 [==============================] - 701s 2ms/step - loss: 0.9250


In [8]:
model_json = model.to_json()
with open("LSTMmodel.json", "w") as json_file:
    json_file.write(model_json)
    
model.save_weights("LSTMmodel.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
data=[]
with open('newMalTest.txt') as f:
    arr = f.readlines()
    content = ''.join(arr)
    items = re.findall("^GET.*|POST.*$", content, re.MULTILINE)
    for item in items:
        temp_data = item.replace(" HTTP/1.1","").lower()
        data.append(temp_data)
probability1=[]

In [47]:
ct=0
j=0
count=[0]*(len(data)+1)
print (len(count))
for seq in data:
    j+=1
    #print (seq)
    probability=[]
    X = numpy.zeros((1,len(seq),VOCAB_SIZE))
    for i in range(1,len(seq),1):
        ix = char_to_ix[seq[i-1]]
        X[0,i,:][ix]=1
    probability = model.predict_on_batch(X)
        #if (model.predict(X[:,:i,:])[0][0][char_to_ix[seq[i]]])<0.00001:
          #count[j]=1
          #print(j)
          #break
        #probability.append(model.predict(X[:,:i,:])[0][0][char_to_ix[seq[i]]])
        #X = numpy.zeros((1,len(seq),VOCAB_SIZE))
    #print(probability)
    for i in range(1,len(seq),1):
        if (probability[0][0][char_to_ix[seq[i]]]<0.00001):
            count[j]=1
            break

for i in count:
  ct+=i
print(ct,j)

1320
1142 1319


In [34]:
print(float(ct)/float(j))
# 1142 1319

0.8658074298711145


In [0]:
#ix = [numpy.random.randint(VOCAB_SIZE)]
#length = 100
#y_char = [ix_to_char[ix[-1]]]
#X = numpy.zeros((1,length,VOCAB_SIZE))
#for i in range(5):
#    X[0,i,:][ix[-1]]=1
#    print(model.predict(X[:,:i+1,:]))